# Load necessary packages

In [1]:
# import of packages
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from random import choices  # To randomly choose clusters
from sklearn.utils import resample 


# autoreload for easier debugging
%load_ext autoreload
%autoreload 2

# Import raw data

In [2]:
# import data
baseline = pd.read_stata('data/baseline.dta')
bok_inflation = pd.read_stata('data/BOK_inflation.dta')
cleanpricedata_y1y2 = pd.read_stata('data/cleanPriceData_Y1Y2.dta')
intensity_obs_short = pd.read_stata('data/intensity_obs_short.dta')
lrfu_select_dataset = pd.read_stata('data/LRFU_select_dataset.dta')
ms1ms2_pooled = pd.read_stata('data/MS1MS2_pooled.dta')
repayment_datay1 = pd.read_stata('data/repayment_dataY1.dta')

# Creating the Tables

## Create Table 1

We start by cleaning the ms1ms2_pooled and baseline data.

In [3]:
# clean ms1ms2_pooled (drop if MS !=2, keep columns oafid and treatMS1MS2, group by oafid and take mean and rename)
ms1ms2_pooled_tab1 = ms1ms2_pooled[ms1ms2_pooled['MS']==2]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1[['oafid', 'treatMS1MS2']]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1.groupby('oafid', as_index=False).mean()
ms1ms2_pooled_tab1.rename(columns={'treatMS1MS2': 'treat13'}, inplace=True)
print(ms1ms2_pooled_tab1.shape[0]) # checking we have the right number of observations as described in the original article

1019


For the baseline data we note that some of the columns have already been renamed with the suffix `_base` however and thus need to account for this. We however, assume that the data have not been altered in any other way compared to what the do in the `do` file.

In [4]:
# clean baseline data (the stata code indicates that the variables columns 'businessprofitmonth' and 'delta' should be kept, however they have already been renamed to 'businessprofitmonth_base' and 'delta_base')
base_cols = ['oafid', 'logtotcons_base', 'male', 'num_adults', 'num_schoolchildren', 'finished_primary',
                   'finished_secondary', 'cropland', 'num_rooms', 'schoolfees', 'totcons_base', 'logpercapcons_base',
                   'total_cash_savings_base', 'total_cash_savings_trimmed', 'has_savings_acct', 'taken_bank_loan',
                   'taken_informal_loan', 'liquidWealth', 'wagepay', 'businessprofitmonth_base', 'price_avg_diff_pct',
                   'price_expect_diff_pct', 'harvest2011', 'netrevenue2011', 'netseller2011', 'autarkic2011',
                   'maizelostpct2011', 'harvest2012', 'correct_interest', 'digit_recall', 'maizegiver', 'delta_base', 'treatment']
baseline_clean = baseline[base_cols].copy()

# rename columns
baseline_clean.columns = [col + '_base' if not col.endswith('_base') and col != 'oafid' and col != 'treatment' else col for col in baseline_clean.columns]
baseline_clean.rename(columns={'treatment': 'treatment2012'}, inplace=True)

# generate treat12 as bool for treatment and control in 2012
baseline_clean['treat12'] = baseline_clean['treatment2012'].apply(lambda x: x in ['T1', 'T2'])
baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_16581/2284489521.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan


Now we can merge the two datasets.

In [5]:
# merge baseline_clean and ms1ms2_pooled_clean on oafid
base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_tab1, on='oafid', how='left')

### Create Table 1.

In [6]:
# create table 1
# copy in case we need this later
df_tab1 = base_ms1ms2_pool.copy()
df_tab1['schoolfees_base'] = df_tab1['schoolfees_base']*1000

# var list for table 1
vars_list = [
    "male_base", "num_adults_base", "num_schoolchildren_base", "finished_primary_base",
    "finished_secondary_base", "cropland_base", "num_rooms_base", "schoolfees_base",
    "totcons_base", "logpercapcons_base", "total_cash_savings_base",
    "total_cash_savings_trimmed_base", "has_savings_acct_base", "taken_bank_loan_base",
    "taken_informal_loan_base", "liquidWealth_base", "wagepay_base",
    "businessprofitmonth_base", "price_avg_diff_pct_base",
    "price_expect_diff_pct_base", "harvest2011_base", "netrevenue2011_base",
    "netseller2011_base", "autarkic2011_base", "maizelostpct2011_base",
    "harvest2012_base", "correct_interest_base", "digit_recall_base",
    "maizegiver_base"
]

renaming = {
    "male_base": "Male",
    "num_adults_base": "Number of adults",
    "num_schoolchildren_base": "Children in school",
    "finished_primary_base": "Finished primary school",
    "finished_secondary_base": "Finished secondary school",
    "cropland_base": "Total cropland (acres)",
    "num_rooms_base": "Number of rooms in household",
    "schoolfees_base": "Total school fees",
    "totcons_base": "Average monthly consumption (Ksh)",
    "logpercapcons_base": "Average monthly consumption/capita (log)",
    "total_cash_savings_base": "Total cash savings (Ksh)",
    "total_cash_savings_trimmed_base": "Total cash savings (trim)",
    "has_savings_acct_base": "Has bank savings acct",
    "taken_bank_loan_base": "Taken bank loan",
    "taken_informal_loan_base": "Taken informal loan",
    "liquidWealth_base": "Liquid wealth (Ksh)",
    "wagepay_base": "Off-farm wages (Ksh)",
    "businessprofitmonth_base": "Business profit (Ksh)",
    "price_avg_diff_pct_base": "Avg $\%\Delta$ price Sep-Jun",
    "price_expect_diff_pct_base": "Expect $\%\Delta$ price Sep12-Jun13",
    "harvest2011_base": "2011 LR harvest (bags)",
    "netrevenue2011_base": "Net revenue 2011 (Ksh)",
    "netseller2011_base": "Net seller 2011",
    "autarkic2011_base": "Autarkic 2011",
    "maizelostpct2011_base": "\% maize lost 2011",
    "harvest2012_base": "2012 LR harvest (bags)",
    "correct_interest_base": "Calculated interest correctly",
    "digit_recall_base": "Digit span recall",
    "maizegiver_base": "Maize giver"
}

# function to perform t-tests
def t_test_by_group(df, var, group_var='treat12'):
    group1 = df[df[group_var] == 0][var].dropna()
    group2 = df[df[group_var] == 1][var].dropna()
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
    return group1.mean(), group2.mean(), len(group1) + len(group2), t_stat, p_val

# applying t-tests and collecting results
results = []
for var in vars_list:
    control_mean, treat_mean, obs, t_stat, p_val = t_test_by_group(df_tab1, var)
    std_diff = (treat_mean - control_mean) / np.sqrt(((len(df_tab1[df_tab1['treat12'] == 0][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 0][var], ddof=1) ** 2 + (len(df_tab1[df_tab1['treat12'] == 1][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 1][var], ddof=1) ** 2) / (len(df_tab1[df_tab1['treat12'] == 0][var]) + len(df_tab1[df_tab1['treat12'] == 1][var]) - 2))
    results.append([var, treat_mean, control_mean, obs, std_diff, p_val])

# convert results to a df to use pandas output to latex
results_df = pd.DataFrame(results, columns=['Variable', 'Treat Mean', 'Control Mean', 'Observations', 'Std Diff', 'P-value'])
results_df['Variable'] = results_df['Variable'].map(renaming)
results_df = results_df.rename(columns={
    'Variable':'Baseline characteristic', 
    'Treat Mean':'Treat', 
    'Control Mean':'Control', 
    'Observations':'Obs', 
    'Std Diff':'Std diff', 
    'P-value':'P-val'})

latex_table1 = results_df.to_latex(index=False, float_format="%.3f")
print(latex_table1)

\begin{tabular}{lrrrrr}
\toprule
Baseline characteristic & Treat & Control & Obs & Std diff & P-val \\
\midrule
Male & 0.296 & 0.334 & 1589 & -0.083 & 0.109 \\
Number of adults & 3.004 & 3.196 & 1510 & -0.099 & 0.067 \\
Children in school & 2.998 & 3.072 & 1589 & -0.038 & 0.454 \\
Finished primary school & 0.718 & 0.772 & 1490 & -0.122 & 0.019 \\
Finished secondary school & 0.253 & 0.270 & 1490 & -0.039 & 0.460 \\
Total cropland (acres) & 2.441 & 2.398 & 1512 & 0.014 & 0.796 \\
Number of rooms in household & 3.073 & 3.252 & 1511 & -0.072 & 0.219 \\
Total school fees & 27239.693 & 29813.631 & 1589 & -0.068 & 0.191 \\
Average monthly consumption (Ksh) & 14970.862 & 15371.378 & 1437 & -0.032 & 0.550 \\
Average monthly consumption/capita (log) & 7.975 & 7.963 & 1434 & 0.019 & 0.721 \\
Total cash savings (Ksh) & 5157.396 & 8021.499 & 1572 & -0.128 & 0.028 \\
Total cash savings (trim) & 4731.623 & 5389.836 & 1572 & -0.050 & 0.343 \\
Has bank savings acct & 0.419 & 0.425 & 1589 & -0.012 & 0.8

## Creating Table 7

In [7]:
# copy the raw data and create columns for treatment and interaction variable
ms1ms2_pooled_tab7 = ms1ms2_pooled.copy()
ms1ms2_pooled_tab7.sort_index(inplace=True)
ms1ms2_pooled_tab7['z'] = pd.NA
ms1ms2_pooled_tab7['z_hi'] = pd.NA

### Running the first set of regressions

In [8]:
# list of treaments
treatments = ['treat12', 'treat13', 'treatMS1MS2']

# list of dependent variables
dependent_vars = ['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']

# list of changeing independent variables depending on the treatment
independent_vars = {
    'treat12': 'Y1round2 + Y1round3',
    'treat13': 'Y2round2 + Y2round3',
    'treatMS1MS2': 'Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3'
    }

# empty dictionary to store results
results = {}
pvals = {var: [] for var in ['z', 'hi', 'z_hi','z+z_hi']}

# Simulating the loop to replace variables and run regressions
for dv in dependent_vars:
    for treat in treatments:
        # Stata automatically omits the missing values in the regression – here we have to do it manually so we copy the data and drop variables
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc','interviewdate'])
        # setting treament variable
        df['z'] = df[treat] # setting z to the treatment variable
        
        # setting interaction variable
        df['z_hi'] = df[treat]*df['hi'] # setting z_hi to the interaction of the treatment hi saturation
        
        # setting the formula to run the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # Run the regression
        i = dependent_vars.index(dv)*len(treatments) + treatments.index(treat)
        results[f'model_{i}'] = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['subloc']})
        # print(results[f'model_{i}'].summary())

        # test the hypothesis that z + z_hi = 0
        hypothesis = 'z + z_hi = 0'
        t_test = results[f'model_{i}'].t_test(hypothesis)

        # store p-value round to 3 decimals
        pvals['z+z_hi'].append(np.round(t_test.pvalue,3))
        pvals['z'].append(np.round(results[f'model_{i}'].pvalues['z'],3))
        pvals['hi'].append(np.round(results[f'model_{i}'].pvalues['hi'],3))
        pvals['z_hi'].append(np.round(results[f'model_{i}'].pvalues['z_hi'],3))

### Running bootstrap regressions

In [15]:
# initialize dictionary to store p-values for each bootstrap iteration
bootstrap_results = {}

n_bootstraps = 2000

# define function to resample within clusters
def resample_within_group(sample_df, group_key):
    grouped = sample_df.groupby('subloc')
    resampled = [resample(group, n_samples=len(group)) for _, group in grouped]
    resampled_df = pd.concat(resampled)
    return resampled_df

# loop over each dependent variable and treatment
for dv in dependent_vars:
    for treat in treatments:
        # copy the raw data and drop missing values
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df.dropna(subset=[dv, treat, 'hi', 'subloc', 'interviewdate'], inplace=True)
        df['subloc'] = df['subloc'].astype(int)
        
        # setting the formula for the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # Update the model key
        i = dependent_vars.index(dv) * len(treatments) + treatments.index(treat)
        model_key = f'model_{i}'
        df['residuals'] = results[model_key].resid

        # Prepare treatment and interaction terms
        df['z'] = df[treat]  # Setting z to the treatment variable
        df['z_hi'] = df[treat] * df['hi']  # Setting z_hi to the interaction of the treatment and hi saturation

        # List unique clusters
        unique_clusters = df['subloc'].unique()
        bootstrap_coeffs = pd.DataFrame(index=range(n_bootstraps), columns=['z', 'hi', 'z_hi'])

        # perform bootstrap iterations
        for j in range(n_bootstraps):
            # resample clusters with replacement
            sampled_clusters = choices(unique_clusters, k=len(unique_clusters))

            # create a new sample DataFrame including all rows belonging to the resampled clusters
            sample = df[df['subloc'].isin(sampled_clusters)]
            resampled_cluster = resample_within_group(sample, 'subloc')

            # perturb residuals and create new dependent variable to follow bootstrap-t procedure with Rademacher weight
            perturbed_residuals = resampled_cluster['residuals'] * np.random.choice([-1, 1], size=len(resampled_cluster['residuals']), replace=True)
            perturbed_y = results[model_key].predict(resampled_cluster) + perturbed_residuals.loc[sample.index]
            perturbed_data = resampled_cluster.assign(y=perturbed_y)

            # run the regression on the perturbed sample
            model = smf.ols(formula, data=perturbed_data).fit()

            # store the coefficients
            bootstrap_coeffs.loc[j] = model.params['z'], model.params['hi'], model.params['z_hi']

            # append the results to the bootstrap_results dictionary
            bootstrap_results[model_key] = bootstrap_coeffs


KeyError: '[9, 15, 16, 18, 19, 24, 25, 30, 38, 168, 174, 192, 200, 279, 281, 289, 296, 297, 301, 302, 356, 357, 383, 393, 525, 531, 534, 535, 540, 542, 600, 612, 646, 648, 653, 667, 668, 672, 673, 745, 751, 756, 757, 766, 770, 774, 795, 797, 810, 812, 826, 830, 843, 845, 847, 848, 949, 1027, 1035, 1037, 1044, 1053, 1077, 1083, 1098, 1108, 1140, 1148, 1149, 1150, 1155, 1157, 1158, 1162, 1163, 1164, 1172, 1174, 1179, 1198, 1205, 1207, 1208, 1223, 1224, 1225, 1232, 1233, 1234, 1245, 1246, 1252, 1256, 1258, 1259, 1261, 1277, 1278, 1285, 1287, 1289, 1290, 1321, 1323, 1334, 1342, 1364, 1369, 1370, 1385, 1388, 1391, 1398, 1405, 1406, 1407, 1408, 1415, 1425, 1436, 1438, 1439, 1454, 1458, 1460, 1461, 1484, 1538, 1554, 1556, 1559, 1562, 1569, 1599, 1601, 1608, 1625, 1647, 1652, 1655, 1656, 1687, 1691, 1692, 1693, 1695, 1696, 1702, 1710, 1711, 1713, 1719, 1720, 1791, 1792, 1795, 1796, 1798, 1887, 1888, 1900, 1906, 1911, 1917, 1922, 1933, 1936, 1963, 1966, 1968, 1976, 2031, 2032, 2038, 2039, 2042,

In [10]:
# create empty dictionary to store bootstrap p-values
bootstrap_pvals = {'z': [], 'hi': [], 'z_hi': []}

# calculate bootstrap p-values for each coefficient
for dv in dependent_vars:
    for treat in treatments:
        model_key = f'model_{dependent_vars.index(dv) * len(treatments) + treatments.index(treat)}'
        original_coefficients = results[model_key].params['z'], results[model_key].params['hi'], results[model_key].params['z_hi']
        extreme_values = (np.abs(bootstrap_results[model_key]) >= np.abs(original_coefficients)).sum(axis=0)
        p_values = extreme_values / n_bootstraps

        for key in bootstrap_pvals.keys():
            bootstrap_pvals[key].append(p_values[key])
        

In [11]:
bootstrap_pvals

{'z': [0.503, 0.4625, 0.4765, 0.5155, 0.4925, 0.526, 0.7615, 0.495, 0.6535],
 'hi': [0.718, 0.8965, 0.938, 0.5335, 0.7435, 0.705, 0.9535, 0.5225, 0.515],
 'z_hi': [0.5365, 0.77, 0.522, 0.5155, 0.525, 0.4865, 0.7725, 0.498, 0.521]}

### Output code to LaTeX

In [12]:
# use stargazer to create a table
result_list = list(results.values())
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.custom_columns(['Inventory', 'Net Revenues', 'Consumption'], [3, 3, 3])
stargazer.rename_covariates({'z': 'Treat', 'hi': 'High', 'z_hi': 'Treat*High'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'hi', 'z_hi'])
# add p-values as a rows 
stargazer.add_line('P-value T + TH = 0', pvals['z+z_hi'])
stargazer.add_line('P-value Treat', pvals['z'])
stargazer.add_line('P-value Treat Bootstrap', bootstrap_pvals['z'])
stargazer.add_line('P-value High', pvals['hi'])
stargazer.add_line('P-value High Bootstrap', bootstrap_pvals['hi'])
stargazer.add_line('P-value Treat*High', pvals['z_hi'])
stargazer.add_line('P-value Treat*High Bootstrap', bootstrap_pvals['z_hi'])


latex_table7 = stargazer.render_latex()

# edit the latex table to add row for telling if Y1 Y2 or Pooled after \\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\
latex_table7 = latex_table7.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \n \\\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\")
latex_table7 = latex_table7.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table7 = latex_table7.replace("Residual Std. Error", "% Residual Std. Error")
latex_table7 = latex_table7.replace("F Statistic", "% F Statistic")
latex_table7 = latex_table7.replace("\\textit{","% \\textit{")

print(latex_table7)

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{3}{c}{Inventory} & \multicolumn{3}{c}{Net Revenues} & \multicolumn{3}{c}{Consumption}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) 
 \\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\
\hline \\[-1.8ex]
 Treat & 0.759$^{***}$ & 0.546$^{***}$ & 0.740$^{***}$ & 1059.602$^{**}$ & 1193.768$^{*}$ & 1101.389$^{**}$ & 0.012$^{}$ & -0.051$^{}$ & -0.011$^{}$ \\
& (0.189) & (0.185) & (0.155) & (437.732) & (685.048) & (430.091) & (0.040) & (0.040) & (0.023) \\
 High & 0.124$^{}$ & -0.028$^{}$ & 0.017$^{}$ & 533.903$^{}$ & -152.603$^{}$ & 164.936$^{}$ & -0.003$^{}$ & -0.084$^{}$ & -0.047$^{}$ \\
& (0.355) & (0.219) & (0.241) & (551.179) & (558.948) & (479.685) & (0.051) & (0.053) & (0.043) \\
 Treat*High & -0.333$^{}$ & -0.065$^{}$ & -0.291$^{}$ & -1114.628$^{**}$ & -555.215$^{}$ & -816.770$^{}$ & -0.013$^{}$ & 0.174$^{***}$